In [1]:
# http://docs.sqlalchemy.org/en/latest/orm/tutorial.html
import sqlalchemy
print(sqlalchemy.__version__)

from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String
class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)
    
    def __repr__(self):
        return f'<user name={self.name}>'
    
Base.metadata.create_all(engine)

1.2.4
2018-03-21 18:01:22,160 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-03-21 18:01:22,161 INFO sqlalchemy.engine.base.Engine ()
2018-03-21 18:01:22,162 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-03-21 18:01:22,162 INFO sqlalchemy.engine.base.Engine ()
2018-03-21 18:01:22,165 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-03-21 18:01:22,165 INFO sqlalchemy.engine.base.Engine ()
2018-03-21 18:01:22,167 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2018-03-21 18:01:22,168 INFO sqlalchemy.engine.base.Engine ()
2018-03-21 18:01:22,169 INFO sqlalchemy.engine.base.Engine COMMIT


In [2]:
from sqlalchemy.orm import sessionmaker, scoped_session
Session = scoped_session(sessionmaker(bind=engine))


In [3]:
# Does Session.commit() will send SQL (No)
Session.commit()

In [4]:
# request NO.1
Session.add(User(name="1"))

print('USER:', Session.query(User).filter(User.name == "1").all())

Session.commit()

print('USER:', Session.query(User).filter(User.name == "1").all())

2018-03-21 18:01:22,199 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-21 18:01:22,202 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2018-03-21 18:01:22,203 INFO sqlalchemy.engine.base.Engine ('1', None, None)
2018-03-21 18:01:22,205 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
2018-03-21 18:01:22,206 INFO sqlalchemy.engine.base.Engine ('1',)
USER: [<user name=1>]
2018-03-21 18:01:22,208 INFO sqlalchemy.engine.base.Engine COMMIT
2018-03-21 18:01:22,210 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-21 18:01:22,211 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
2018-03-21 18:01:22,212 INFO sqlalchemy.engine.base.Engine ('1',)


In [5]:
# request NO.2

session2 = Session()
session3 = Session()

session2.add(User(name="2"))
print('USER:', session3.query(User).filter(User.name == "2").all())
session2.commit()


2018-03-21 18:01:22,225 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2018-03-21 18:01:22,227 INFO sqlalchemy.engine.base.Engine ('2', None, None)
2018-03-21 18:01:22,228 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
2018-03-21 18:01:22,229 INFO sqlalchemy.engine.base.Engine ('2',)
USER: [<user name=2>]
2018-03-21 18:01:22,230 INFO sqlalchemy.engine.base.Engine COMMIT


In [6]:
try:
    session4 = Session()
    session4.add(User(name="4"))
    
    assert 1==2
    session4.commit()
except Exception as e:
    Session.rollback()
    
print(session4.query(User).filter(User.name == "4").all())
    

2018-03-21 18:01:22,252 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-21 18:01:22,254 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
2018-03-21 18:01:22,254 INFO sqlalchemy.engine.base.Engine ('4',)
[]
